In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    DDD_Train_Decile8 = pd.read_pickle("DDD_Train_Decile8.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile8.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile8, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150,200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 8')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 5s 2ms/step - loss: 0.3636 - acc: 0.8489 - val_loss: 0.3077 - val_acc: 0.8854
Epoch 39/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3580 - acc: 0.8600 - val_loss: 0.3076 - val_acc: 0.8818
Epoch 40/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3599 - acc: 0.8525 - val_loss: 0.3088 - val_acc: 0.8854
Epoch 41/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3586 - acc: 0.8542 - val_loss: 0.3086 - val_acc: 0.8871

Epoch 00041: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 42/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3567 - acc: 0.8564 - val_loss: 0.3058 - val_acc: 0.8924
Epoch 43/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3595 - acc: 0.8595 - val_loss: 0.3051 - val_acc: 0.8871
Epoch 44/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3537 - acc: 0.8560 - val_loss: 0.3041 - val_acc: 0.8

Epoch 98/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3499 - acc: 0.8582 - val_loss: 0.2980 - val_acc: 0.8942
Epoch 99/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3570 - acc: 0.8503 - val_loss: 0.2978 - val_acc: 0.8924
Epoch 100/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3471 - acc: 0.8635 - val_loss: 0.2977 - val_acc: 0.8942
Test accuracy: 0.8941798938645257
Train on 2264 samples, validate on 567 samples
Epoch 1/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.6336 - acc: 0.5892 - val_loss: 0.5621 - val_acc: 0.6402
Epoch 2/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.5045 - acc: 0.7602 - val_loss: 0.4159 - val_acc: 0.8307
Epoch 3/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4354 - acc: 0.8419 - val_loss: 0.3629 - val_acc: 0.8377
Epoch 4/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3851 - acc: 0.8476 - val_l

2264/2264 [==============================] - 5s 2ms/step - loss: 0.3205 - acc: 0.8595 - val_loss: 0.2692 - val_acc: 0.9048
Epoch 57/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3184 - acc: 0.8653 - val_loss: 0.2693 - val_acc: 0.9048
Epoch 58/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3164 - acc: 0.8609 - val_loss: 0.2694 - val_acc: 0.9012
Epoch 59/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3226 - acc: 0.8564 - val_loss: 0.2692 - val_acc: 0.9048
Epoch 60/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3188 - acc: 0.8578 - val_loss: 0.2692 - val_acc: 0.9012
Epoch 61/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3128 - acc: 0.8617 - val_loss: 0.2690 - val_acc: 0.9048
Epoch 62/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3158 - acc: 0.8644 - val_loss: 0.2689 - val_acc: 0.9048
Epoch 63/150
2264/2264 [==============================] - 5s 

2264/2264 [==============================] - 5s 2ms/step - loss: 0.3207 - acc: 0.8582 - val_loss: 0.2687 - val_acc: 0.8995
Epoch 117/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3242 - acc: 0.8542 - val_loss: 0.2691 - val_acc: 0.8995
Epoch 118/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3239 - acc: 0.8587 - val_loss: 0.2697 - val_acc: 0.9012
Epoch 119/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3172 - acc: 0.8635 - val_loss: 0.2698 - val_acc: 0.9012
Epoch 120/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3205 - acc: 0.8573 - val_loss: 0.2700 - val_acc: 0.8995
Epoch 121/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3183 - acc: 0.8609 - val_loss: 0.2695 - val_acc: 0.9012
Epoch 122/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3198 - acc: 0.8582 - val_loss: 0.2694 - val_acc: 0.9012
Epoch 123/150
2264/2264 [==============================

...

2264/2264 [==============================] - 5s 2ms/step - loss: 0.3511 - acc: 0.8481 - val_loss: 0.3012 - val_acc: 0.8818
Epoch 68/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3585 - acc: 0.8542 - val_loss: 0.3014 - val_acc: 0.8854
Epoch 69/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3502 - acc: 0.8547 - val_loss: 0.3012 - val_acc: 0.8836
Epoch 70/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3547 - acc: 0.8503 - val_loss: 0.2980 - val_acc: 0.8836
Epoch 71/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3493 - acc: 0.8516 - val_loss: 0.2970 - val_acc: 0.8854
Epoch 72/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3460 - acc: 0.8569 - val_loss: 0.2974 - val_acc: 0.8854
Epoch 73/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3491 - acc: 0.8534 - val_loss: 0.2982 - val_acc: 0.8854
Epoch 74/100
2264/2264 [==============================] - 5s 

Epoch 26/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3175 - acc: 0.8622 - val_loss: 0.2693 - val_acc: 0.9030
Epoch 27/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3179 - acc: 0.8595 - val_loss: 0.2692 - val_acc: 0.9012

Epoch 00027: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 28/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3200 - acc: 0.8578 - val_loss: 0.2693 - val_acc: 0.9012
Epoch 29/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3222 - acc: 0.8613 - val_loss: 0.2693 - val_acc: 0.9012
Epoch 30/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3155 - acc: 0.8648 - val_loss: 0.2692 - val_acc: 0.9012
Epoch 31/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3216 - acc: 0.8587 - val_loss: 0.2695 - val_acc: 0.9012
Epoch 32/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3177 - acc: 0.8613 - v

2264/2264 [==============================] - 6s 3ms/step - loss: 0.3270 - acc: 0.8573 - val_loss: 0.2694 - val_acc: 0.8995
Epoch 86/200
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3146 - acc: 0.8631 - val_loss: 0.2695 - val_acc: 0.8995
Epoch 87/200
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3122 - acc: 0.8595 - val_loss: 0.2694 - val_acc: 0.8995
Epoch 88/200
2264/2264 [==============================] - 7s 3ms/step - loss: 0.3199 - acc: 0.8613 - val_loss: 0.2692 - val_acc: 0.8995
Epoch 89/200
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3178 - acc: 0.8626 - val_loss: 0.2693 - val_acc: 0.8995
Epoch 90/200
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3195 - acc: 0.8622 - val_loss: 0.2692 - val_acc: 0.8995
Epoch 91/200
2264/2264 [==============================] - 7s 3ms/step - loss: 0.3229 - acc: 0.8587 - val_loss: 0.2693 - val_acc: 0.8977
Epoch 92/200
2264/2264 [==============================] - 6s 

2264/2264 [==============================] - 6s 3ms/step - loss: 0.3225 - acc: 0.8573 - val_loss: 0.2690 - val_acc: 0.8995
Epoch 146/200
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3175 - acc: 0.8538 - val_loss: 0.2691 - val_acc: 0.9030
Epoch 147/200
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3206 - acc: 0.8609 - val_loss: 0.2692 - val_acc: 0.8995
Epoch 148/200
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3189 - acc: 0.8595 - val_loss: 0.2692 - val_acc: 0.8995
Epoch 149/200
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3176 - acc: 0.8591 - val_loss: 0.2692 - val_acc: 0.8995
Epoch 150/200
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3189 - acc: 0.8595 - val_loss: 0.2691 - val_acc: 0.8995
Epoch 151/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3135 - acc: 0.8573 - val_loss: 0.2689 - val_acc: 0.8995
Epoch 152/200
2264/2264 [==============================

2264/2264 [==============================] - 6s 3ms/step - loss: 0.4342 - acc: 0.8485 - val_loss: 0.3757 - val_acc: 0.8713
Epoch 5/50
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4063 - acc: 0.8529 - val_loss: 0.3429 - val_acc: 0.8660
Epoch 6/50
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3825 - acc: 0.8511 - val_loss: 0.3152 - val_acc: 0.8818
Epoch 7/50
2264/2264 [==============================] - 7s 3ms/step - loss: 0.3635 - acc: 0.8525 - val_loss: 0.3315 - val_acc: 0.8413
Epoch 8/50
2264/2264 [==============================] - 7s 3ms/step - loss: 0.3575 - acc: 0.8472 - val_loss: 0.2972 - val_acc: 0.8959
Epoch 9/50
2264/2264 [==============================] - 7s 3ms/step - loss: 0.3565 - acc: 0.8582 - val_loss: 0.3021 - val_acc: 0.8959
Epoch 10/50
2264/2264 [==============================] - 6s 3ms/step - loss: 0.3475 - acc: 0.8573 - val_loss: 0.2976 - val_acc: 0.8889
Epoch 11/50
2264/2264 [==============================] - 6s 3ms/step - l

In [ ]:
0.86434